In [1]:
from src.data_loader import *
from torch.utils.data import DataLoader
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Real Data Testing
# Data Loader

dataset = 0

if dataset:

    path = '../data/AudioMNIST_Indicies/dummy_labels.csv'
    root_dir = '../data/AudioMNIST/'

else:
    path = '../data/AvianID_AcousticIndices/UK_AI.csv'
    root_dir='../data/UK_BIRD/'

# 

ds = AvianNatureSounds(annotation_file_path=path,
                       root_dir=root_dir,
                       key='habitat',
                       mode='stft',
                       length=5,
                       sampling_rate=44100,
                       n_fft=1024,
                       hop_length=512,
                       mel_spectrogram=None,
                       verbose=False,
                       fixed_limit=True)



train_loader = DataLoader(dataset=ds, batch_size=4, shuffle=True)



batch = next(iter(train_loader))

In [2]:
from utils.plot_spectrograms import plot_spectrograms

# plot_spectrograms(batch=batch[0],magnitude=False) # magnitude=True , phase = False

In [3]:
def convert_to_complex(mag, angle):
    # https://dsp.stackexchange.com/a/72172 
    return mag*(np.cos(angle)+1j* np.sin(angle))

In [4]:
# print(batch[0][0].shape)
complex_spectrogram = convert_to_complex(batch[0][0][0], batch[0][0][1])

# print(complex_spectrogram.shape)
new_row = torch.zeros(1, 512)

# print(new_row.shape)
complex_spectrogram = torch.cat((complex_spectrogram, new_row), dim=0)

# print(complex_spectrogram.shape)
signal = torch.istft(complex_spectrogram,n_fft=1024,hop_length=512)
signal.unsqueeze_(0).unsqueeze_(0)
print(signal.shape)

torch.Size([1, 1, 261632])


In [14]:
signals = []

for i in range(4):
    complex_spectrogram = convert_to_complex(batch[0][i][0], batch[0][i][1])
    new_row = torch.zeros(1, 512)
    complex_spectrogram = torch.cat((complex_spectrogram, new_row), dim=0)
    signal = torch.istft(complex_spectrogram,n_fft=1024,hop_length=512).unsqueeze_(0).unsqueeze_(0)
    signals.append(signal)

signals = torch.cat(signals, dim=0)

print(signals.shape)


lin_lay = nn.Linear(128*128, 65385)
conv = nn.Conv2d(1, 32, kernel_size=5, stride=2,padding=2)
conv2 = nn.Conv2d(32, 32, kernel_size=5, stride=2,padding=2)


conv1d = nn.Conv1d(1, 32, kernel_size=32, stride=2)
conv1d32 = nn.Conv1d(32, 32, kernel_size=32, stride=2)


# print(lin_lay(signals.view(4, -1)).shape)

print(conv1d32(conv1d(signals)).shape)
phase_out = conv1d32(conv1d(signals))

batch[1].shape

# print(conv(conv(batch[1])).shape)
mag_out = conv2(conv(batch[1]))

print('#### LOCK HERE ####')
print(mag_out.shape)
print('###################')

mag_lin_out = lin_lay(mag_out.view(4, -1))

torch.cat([mag_lin_out.unsqueeze_(1), phase_out], dim=1).shape

torch.Size([4, 1, 261632])
torch.Size([4, 32, 65385])
#### LOCK HERE ####
torch.Size([4, 32, 128, 128])
###################


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x524288 and 16384x65385)

In [6]:
batch[1].shape

# print(conv(conv(batch[1])).shape)
mag_out = conv(conv(batch[1]))

mag_lin_out = lin_lay(mag_out.view(4, -1))

torch.cat([mag_lin_out.unsqueeze_(1), phase_out], dim=1).shape







torch.Size([4, 33, 65385])

In [15]:
128 * 128 * 32

524288

In [17]:
torch.randn(2,32,16,16).view(2, -1).shape

torch.Size([2, 8192])

In [21]:
t = torch.randn(2,32,16,16)

t.view(2,-1).shape

torch.Size([2, 8192])